In [1]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
import IPython.display as ipd
from PIL import Image
import os
import scipy.signal as signal
from tqdm import tqdm
import random

In [2]:
# Step 1: Denoising using a low pass filter
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [3]:
# Downsampling audio
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [4]:
# Split audio into fixed-length segments
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [5]:
def norm_mfcc(audio_path):

    # Load audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Compute MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=48)

    # Normalize MFCC values
    normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
    return normalized_mfcc

In [6]:
def time_warp(mfcc, time_warping_range):
    num_frames = mfcc.shape[1]
    t = np.arange(num_frames)
    random_shift = np.random.randint(-time_warping_range, time_warping_range)
    warped_t = np.clip(t + random_shift, 0, num_frames - 1)
    warped_mfcc = mfcc[:, warped_t]
    return warped_mfcc[:, :48]

In [7]:
def frequency_mask(mfcc, frequency_masking_para, mask_max_frames):
    num_mfcc = mfcc.shape[0]
    num_frames = mfcc.shape[1]
    f = np.random.randint(0, frequency_masking_para)
    f0 = random.randint(0, num_mfcc - f)
    mfcc[f0:f0 + f, :] = 0.0
    return mfcc[:, :48]

In [8]:
Df_A=pd.read_csv("../dataframes/Heart_Audio_Default_Folders_A.csv")

In [9]:
Df_A

,File,Class,Folder
0,201012172012.wav,Artifact,Atraining_artifact
1,201105040918.wav,Artifact,Atraining_artifact
2,201105041959.wav,Artifact,Atraining_artifact
3,201105051017.wav,Artifact,Atraining_artifact
4,201105060108.wav,Artifact,Atraining_artifact
...,...,...,...
119,201103200218.wav,Extrasystole,Atraining_extrahls
120,201104021355.wav,Extrasystole,Atraining_extrahls
121,201104140118.wav,Extrasystole,Atraining_extrahls
122,201104270458.wav,Extrasystole,Atraining_extrahls


In [10]:
input_shape = (39,32)  # Example input shape for MobileNetV2

In [11]:
MFCC_Dataset=[]
folder_path = "../Datasets/A_data/time_shifted_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [12]:
len(MFCC_Dataset)

291

In [13]:
folder_path = "../Datasets/A_data/pitch_shifted_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [14]:
len(MFCC_Dataset)

582

In [15]:
folder_path ="../Datasets/A_data/normalised_audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    
    # Set time warping range (adjust according to your needs)
    time_warping_range = 5

    warped_mfcc = time_warp(mfcc_array, time_warping_range)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(warped_mfcc,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    
    # Set frequency masking parameters (adjust according to your needs)
    frequency_masking_para = 5
    mask_max_frames = 5

    masked_mfcc = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
    masked_mfcc2 = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(masked_mfcc, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    mfcc_reshaped = np.reshape(masked_mfcc2, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [16]:
len(MFCC_Dataset)

1746

In [17]:
MFCC_Df=pd.DataFrame(MFCC_Dataset, columns=['MFCC', 'Class'])

In [18]:
MFCC_Df

,MFCC,Class
0,"[[-5.269797, -5.763967, -6.6692266, -6.3825703...",Extrasystole
1,"[[-5.6569643, -6.1762, -6.7199655, -6.6153536,...",Extrasystole
2,"[[-3.1142056, -3.6126595, -6.192634, -6.212791...",Artifact
3,"[[-3.2704666, -3.6076217, -5.9305277, -6.04547...",Artifact
4,"[[-6.0625944, -6.4801135, -7.0279455, -6.63700...",Normal
...,...,...
1741,"[[-6.264359, -6.0892177, -6.101441, -6.2145867...",Murmur
1742,"[[-7.60592, -7.627124, -7.6543207, -7.6105256,...",Artifact
1743,"[[-7.627124, -7.6543207, -7.6105256, -7.49417,...",Artifact
1744,"[[-7.60592, -7.627124, -7.6543207, -7.6105256,...",Artifact


In [19]:
MFCC_Df['Class'].value_counts()

Class
Artifact        720
Normal          402
Murmur          402
Extrasystole    222
Name: count, dtype: int64

In [20]:
for i in MFCC_Df["MFCC"]:
    print(i.shape)

(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(

In [21]:
# Define the column value to filter
column_value = 'Artifact'

# Filter the DataFrame based on the column value
filtered_df = MFCC_Df[MFCC_Df['Class'] == column_value]

# Calculate the number of rows to drop (half of the filtered rows)
rows_to_drop = len(filtered_df) // 2

# Randomly select rows to drop
rows = np.random.choice(filtered_df.index, rows_to_drop, replace=False)

# Drop the selected rows
MFCC_Final = MFCC_Df.drop(rows)

# Print the updated DataFrame
print(MFCC_Final)

                                                   MFCC         Class
0     [[-5.269797, -5.763967, -6.6692266, -6.3825703...  Extrasystole
1     [[-5.6569643, -6.1762, -6.7199655, -6.6153536,...  Extrasystole
2     [[-3.1142056, -3.6126595, -6.192634, -6.212791...      Artifact
3     [[-3.2704666, -3.6076217, -5.9305277, -6.04547...      Artifact
4     [[-6.0625944, -6.4801135, -7.0279455, -6.63700...        Normal
...                                                 ...           ...
1738  [[-6.264359, -6.0892177, -6.101441, -6.2145867...        Murmur
1739  [[-6.412322, -6.758966, -6.60008, -6.3091345, ...        Murmur
1740  [[-6.264359, -6.0892177, -6.101441, -6.2145867...        Murmur
1741  [[-6.264359, -6.0892177, -6.101441, -6.2145867...        Murmur
1743  [[-7.627124, -7.6543207, -7.6105256, -7.49417,...      Artifact

[1386 rows x 2 columns]


In [22]:
MFCC_Final['Class'].value_counts()

Class
Normal          402
Murmur          402
Artifact        360
Extrasystole    222
Name: count, dtype: int64

In [23]:
X=np.array(MFCC_Final["MFCC"].tolist())
y=np.array(MFCC_Final["Class"].tolist())

In [24]:
X.shape

(1386, 39, 32)

In [25]:
input_shape = (39, 32, 3)
X = np.repeat(X[..., np.newaxis], 3, axis=-1)

In [26]:
np.unique(y)

array(['Artifact', 'Extrasystole', 'Murmur', 'Normal'], dtype='<U12')

In [27]:
y=np.array(pd.get_dummies(y))

In [28]:
y

array([[False,  True, False, False],
       [False,  True, False, False],
       [ True, False, False, False],
       ...,
       [False, False,  True, False],
       [False, False,  True, False],
       [ True, False, False, False]])

In [29]:
X.shape

(1386, 39, 32, 3)

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
X_train.shape

(1108, 39, 32, 3)

In [32]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [33]:
X_train.shape

(941, 39, 32, 3)

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D,MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [35]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained ResNet152V2 model
resnet152v2 = ResNet152V2(weights='imagenet', include_top=False, input_shape=(39, 32, 3))

# Freeze the pre-trained layers
for layer in resnet152v2.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(resnet152v2.output)
dense_layer = Dense(128, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.35)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=resnet152v2.input, outputs=classification_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Metal device set to: Apple M1 Pro
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 39, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 45, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 20, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 22, 18, 64)           0         ['conv1_conv[0][0]']          
                                                            

 tchNormalization)                                                                                
                                                                                                  
 conv2_block3_preact_relu (  (None, 10, 8, 256)           0         ['conv2_block3_preact_bn[0][0]
 Activation)                                                        ']                            
                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 10, 8, 64)            16384     ['conv2_block3_preact_relu[0][
 D)                                                                 0]']                          
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 10, 8, 64)            256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
          

 conv3_block2_1_relu (Activ  (None, 5, 4, 128)            0         ['conv3_block2_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block2_2_pad (ZeroPa  (None, 7, 6, 128)            0         ['conv3_block2_1_relu[0][0]'] 
 dding2D)                                                                                         
                                                                                                  
 conv3_block2_2_conv (Conv2  (None, 5, 4, 128)            147456    ['conv3_block2_2_pad[0][0]']  
 D)                                                                                               
                                                                                                  
 conv3_block2_2_bn (BatchNo  (None, 5, 4, 128)            512       ['conv3_block2_2_conv[0][0]'] 
 rmalizati

                                                                                                  
 conv3_block4_out (Add)      (None, 5, 4, 512)            0         ['conv3_block3_out[0][0]',    
                                                                     'conv3_block4_3_conv[0][0]'] 
                                                                                                  
 conv3_block5_preact_bn (Ba  (None, 5, 4, 512)            2048      ['conv3_block4_out[0][0]']    
 tchNormalization)                                                                                
                                                                                                  
 conv3_block5_preact_relu (  (None, 5, 4, 512)            0         ['conv3_block5_preact_bn[0][0]
 Activation)                                                        ']                            
                                                                                                  
 conv3_blo

 ation)                                                                                           
                                                                                                  
 conv3_block7_2_pad (ZeroPa  (None, 7, 6, 128)            0         ['conv3_block7_1_relu[0][0]'] 
 dding2D)                                                                                         
                                                                                                  
 conv3_block7_2_conv (Conv2  (None, 5, 4, 128)            147456    ['conv3_block7_2_pad[0][0]']  
 D)                                                                                               
                                                                                                  
 conv3_block7_2_bn (BatchNo  (None, 5, 4, 128)            512       ['conv3_block7_2_conv[0][0]'] 
 rmalization)                                                                                     
          

...

 vation)                                                                                          
                                                                                                  
 conv4_block29_2_pad (ZeroP  (None, 5, 4, 256)            0         ['conv4_block29_1_relu[0][0]']
 adding2D)                                                                                        
                                                                                                  
 conv4_block29_2_conv (Conv  (None, 3, 2, 256)            589824    ['conv4_block29_2_pad[0][0]'] 
 2D)                                                                                              
                                                                                                  
 conv4_block29_2_bn (BatchN  (None, 3, 2, 256)            1024      ['conv4_block29_2_conv[0][0]']
 ormalization)                                                                                    
          

 conv4_block31_out (Add)     (None, 3, 2, 1024)           0         ['conv4_block30_out[0][0]',   
                                                                     'conv4_block31_3_conv[0][0]']
                                                                                                  
 conv4_block32_preact_bn (B  (None, 3, 2, 1024)           4096      ['conv4_block31_out[0][0]']   
 atchNormalization)                                                                               
                                                                                                  
 conv4_block32_preact_relu   (None, 3, 2, 1024)           0         ['conv4_block32_preact_bn[0][0
 (Activation)                                                       ]']                           
                                                                                                  
 conv4_block32_1_conv (Conv  (None, 3, 2, 256)            262144    ['conv4_block32_preact_relu[0]
 2D)      

                                                                                                  
 conv4_block34_2_pad (ZeroP  (None, 5, 4, 256)            0         ['conv4_block34_1_relu[0][0]']
 adding2D)                                                                                        
                                                                                                  
 conv4_block34_2_conv (Conv  (None, 3, 2, 256)            589824    ['conv4_block34_2_pad[0][0]'] 
 2D)                                                                                              
                                                                                                  
 conv4_block34_2_bn (BatchN  (None, 3, 2, 256)            1024      ['conv4_block34_2_conv[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv4_blo

 2D)                                                                                              
                                                                                                  
 conv4_block36_out (Add)     (None, 2, 1, 1024)           0         ['max_pooling2d_2[0][0]',     
                                                                     'conv4_block36_3_conv[0][0]']
                                                                                                  
 conv5_block1_preact_bn (Ba  (None, 2, 1, 1024)           4096      ['conv4_block36_out[0][0]']   
 tchNormalization)                                                                                
                                                                                                  
 conv5_block1_preact_relu (  (None, 2, 1, 1024)           0         ['conv5_block1_preact_bn[0][0]
 Activation)                                                        ']                            
          

 conv5_block3_1_bn (BatchNo  (None, 2, 1, 512)            2048      ['conv5_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_1_relu (Activ  (None, 2, 1, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_pad (ZeroPa  (None, 4, 3, 512)            0         ['conv5_block3_1_relu[0][0]'] 
 dding2D)                                                                                         
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 2, 1, 512)            2359296   ['conv5_block3_2_pad[0][0]']  
 D)       

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
checkpoint = ModelCheckpoint('heart_resnet_mfcc.hdf5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[checkpoint])

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 1.0733 - accuracy: 0.5356

/Users/atrix/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 18s 320ms/step - loss: 1.0733 - accuracy: 0.5356 - val_loss: 0.7258 - val_accuracy: 0.7186
Epoch 2/50
30/30 [==============================] - 3s 89ms/step - loss: 0.7452 - accuracy: 0.6854 - val_loss: 0.6384 - val_accuracy: 0.7006
Epoch 3/50
30/30 [==============================] - 3s 105ms/step - loss: 0.6358 - accuracy: 0.7311 - val_loss: 0.5603 - val_accuracy: 0.7605
Epoch 4/50
30/30 [==============================] - 3s 106ms/step - loss: 0.5694 - accuracy: 0.7630 - val_loss: 0.4803 - val_accuracy: 0.8084
Epoch 5/50
30/30 [==============================] - 3s 99ms/step - loss: 0.5362 - accuracy: 0.7598 - val_loss: 0.4655 - val_accuracy: 0.8383
Epoch 6/50
30/30 [==============================] - 2s 78ms/step - loss: 0.4804 - accuracy: 0.8023 - val_loss: 0.4733 - val_accuracy: 0.8323
Epoch 7/50
30/30 [==============================] - 3s 104ms/step - loss: 0.4298 - accuracy: 0.8385 - val_loss: 0.3975 - val_accuracy: 0.8563
Epoch 8/50
30/30 [=

In [38]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9208633303642273


In [42]:
from tensorflow import keras

In [39]:
def pred(audio_path, model_path):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)
    
    model = keras.models.load_model(model_path)
    i=0
    
    for segment in segments:
        # Compute MFCC features
        mfcc = librosa.feature.mfcc(y=segment, sr=target_sampling_rate, n_mfcc=48)

        # Normalize MFCC values
        normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        mfcc_reshaped = np.reshape(normalized_mfcc,(1,39,32))
        
        input_shape = (39, 32, 3)
        mfcc_final = np.repeat(mfcc_reshaped[..., np.newaxis], 3, axis=-1)
        
        prediction = model.predict(mfcc_final)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        i=i+1

#         prediction = model.predict(img_arr)
#         x=np.argmax(prediction)

        classes={0:'Artifact', 1:'Extrasystole', 2:'Murmur', 3:'Normal'}
        print(classes[x],confidence)
#         print(mfcc_final.shape)

In [43]:
audio_path= "../Datasets/A_data/Atraining_extrahls/201101160808.wav"
model_path= "heart_resnet_mfcc.hdf5"
pred(audio_path,model_path)

1/1 [==============================] - 4s 4s/step
Extrasystole 0.9851873
1/1 [==============================] - 0s 255ms/step
Extrasystole 0.72692937


In [44]:
import tensorflow.keras.backend as K

In [45]:
K.clear_session()